In [1]:
import json, requests
from bs4 import BeautifulSoup


In [2]:
URLS = [
    "https://www.esilv.fr/",
    "https://www.esilv.fr/lecole/",
    "https://www.esilv.fr/admissions/",
    "https://www.esilv.fr/formations/cycle-ingenieur/",

]

In [3]:
def extract(url):
    try:
        html = requests.get(url, timeout=15).text  # télécharge le contenu HTML de la page
    except:
        return None
    soup = BeautifulSoup(html, "html.parser")      # parse le HTML pour le rendre lisible
    for tag in soup(["header", "footer", "nav", "script", "style"]):
        tag.decompose()                            # supprime les éléments inutiles (menus, scripts…)
    title = (soup.title.get_text(strip=True) if soup.title else "").strip()  # récupère le titre
    text = soup.get_text("\n", strip=True)         # extrait tout le texte visible
    if len(text) < 200:                            # ignore les pages trop courtes
        return None
    return {"url": url, "title": title, "text": text}  # retourne les infos sous forme de dictionnaire

with open("esilv_pages.jsonl", "w", encoding="utf-8") as f:
    for url in URLS:
        doc = extract(url)
        if doc:
            f.write(json.dumps(doc, ensure_ascii=False) + "\n")


In [4]:
# fonction d’extraction d’une page
def extract(url):
    try:
        html = requests.get(url, timeout=15).text  # télécharge la page
    except:
        return None
    soup = BeautifulSoup(html, "html.parser")      # parse le HTML
    for tag in soup(["header", "footer", "nav", "script", "style"]):
        tag.decompose()                            # supprime les parties inutiles
    title = (soup.title.get_text(strip=True) if soup.title else "").strip()
    text = soup.get_text("\n", strip=True)
    if len(text) < 200:
        return None
    return {"url": url, "title": title, "text": text}

#utliser les robots pour scrapper

In [ ]:
# récupère toutes les URLs depuis le sitemap principal
def get_all_urls(main_sitemap):
    html = requests.get(main_sitemap, timeout=15).text
    soup = BeautifulSoup(html, "xml")
    sitemap_links = [loc.get_text() for loc in soup.find_all("loc")]
    all_urls = []
    for sitemap in sitemap_links:
        xml = requests.get(sitemap, timeout=15).text
        sub = BeautifulSoup(xml, "xml")
        all_urls += [loc.get_text() for loc in sub.find_all("loc")]
    return all_urls

# lit toutes les URLs du site via le sitemap principal
URLS = get_all_urls("https://www.esilv.fr/sitemap_index.xml")

with open("esilv_all_pages.jsonl", "w", encoding="utf-8") as f:
    for url in URLS:
        doc = extract(url)
        if doc:
            f.write(json.dumps(doc, ensure_ascii=False) + "\n")
